In [ ]:
import sys
root_dir = '../../'
sys.path.append(root_dir)

In [1]:
import pickle
from utils import *

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Load data

In [3]:
lorenz = pd.read_csv(root_dir + '/data/lorenz.csv', header=None).iloc[:, 0].values
X_train = lorenz[4_000: 24_000]
Y_true = lorenz[24_000: 26_000]

## Multiple trajectories: Predictions using random perturbations and clustering of noisy trajectories

In [4]:
# get patterns and motifs

patterns = get_patterns(
    pattern_size = 3, 
    max_distance = 10, 
    patterns_percent = 100
)

motifs = get_motifs(X_train, patterns)

In [5]:
trajectories_num = 30
trajectory_length = 100
n = 96 * 3   # number of iterations (experiments)

# prepare inputs for get_multiple_trajectories() on each iteration
inputs = []
for i in range(n):
    _input = (
        trajectories_num,
        trajectory_length, 
        np.append(X_train, Y_true[:i]), 
        motifs
    )   
    inputs.append(_input)


def task(_input):
    trajectories_num, trajectory_length, _X_train, motifs = _input
    
    multiple_trajectories = get_multiple_trajectories(
        trajectories_num = trajectories_num,
        trajectory_length = trajectory_length, 
        X_train = _X_train, 
        motifs = motifs, 
        distance_eps = 0.01, 
        cluster_eps = 0.01, 
        min_samples = 5,
        cluster_1_proportion_threshold = -1, 
        clusters_diff_threshold = -1, 
        sigma = 0.01
    )

    return multiple_trajectories


results = run_concurrently(task, inputs)

In [ ]:

with open('results-base.pickle', 'wb') as f:
    pickle.dump(results, f)